<a href="https://colab.research.google.com/github/WooML/sat_solving/blob/main/%EB%AF%B8%EB%8B%88%EC%BA%A1%EC%8A%A4%ED%86%A4(%EC%98%81%EC%96%B4%EB%AC%B8%EC%A0%9C_%ED%92%80%EC%9D%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BERT 적용(인터넷)

In [ ]:
!pip install dill

In [ ]:
import torchtext

!pip install --upgrade torchtext==0.6.0

In [ ]:
import dill
from copy import deepcopy
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc

import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn

from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import BucketIterator
from torchtext.data import Iterator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
RANDOM_SEED = 2020
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# DATA_PATH = "data/processed/"
DATA_PATH = "/content/drive/MyDrive/미니캡스톤 자료"

In [ ]:
import pandas as pd

dataset = pd.read_csv("/content/drive/MyDrive/미니캡스톤 자료/sat_train1.tsv",delimiter='\t')

print(dataset)

                                              context  label
0   Competitive activities can be more than just p...      0
1   The provision of timely, constructive feedback...      1
2         In a sense, all competitions give feedback.      1
3   For many, this is restricted to information ab...      1
4   The provision of that type of feedback can be ...      1
5   The best competitions promote excellence, not ...      1
6   The emphasis on superiority is what we typical...      1
7   Performance feedback requires that the program...      1
8   Information about performance can be very help...      1
9   People from more individualistic cultural cont...      1
10  With this form of agency comes the belief that...      0
11  The independent self may be more 3 driven to c...      1
12  However, people from more interdependent cultu...      1
13  Research has shown 4 that East Asians prefer t...      1
14  Therefore, people 5 who hold a more interdepen...      1


##학습

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd

# 데이터셋 클래스 정의
class GrammarDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터 준비
grammer_dataset = pd.read_csv("/content/drive/MyDrive/미니캡스톤 자료/sat_train1.tsv",delimiter='\t')
sentences = grammer_dataset['context']
labels = grammer_dataset['label']

# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 데이터셋 및 데이터로더 생성
max_length = 128
batch_size = 2
dataset = GrammarDataset(sentences, labels, tokenizer, max_length)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=1e-5)

# 모델 학습
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss}")

# 모델 저장 경로 정의
model_save_path = "/content/drive/MyDrive/미니캡스톤 자료/bert_grammar_model(20)두번째.pth"

# 모델 저장
torch.save(model.state_dict(), model_save_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 4.074541747570038
Epoch 2, Loss: 3.175699234008789
Epoch 3, Loss: 3.258248656988144
Epoch 4, Loss: 2.207800179719925
Epoch 5, Loss: 2.0708748251199722
Epoch 6, Loss: 1.8040107786655426
Epoch 7, Loss: 1.4440930783748627
Epoch 8, Loss: 1.169882643967867
Epoch 9, Loss: 0.9084118418395519
Epoch 10, Loss: 0.7389557287096977
Epoch 11, Loss: 0.5487607717514038
Epoch 12, Loss: 0.4430824723094702
Epoch 13, Loss: 0.3767631109803915
Epoch 14, Loss: 0.3182284813374281
Epoch 15, Loss: 0.3300452046096325
Epoch 16, Loss: 0.26217636466026306
Epoch 17, Loss: 0.21442247182130814
Epoch 18, Loss: 0.23558611050248146
Epoch 19, Loss: 0.23586584441363811
Epoch 20, Loss: 0.2721232511103153


##테스트

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [ ]:
# 모델 저장 경로
model_save_path = '/content/drive/MyDrive/미니캡스톤 자료/bert_grammar_model(20)두번째.pth'

# 모델 로드
loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.eval()

# 테스트 문장 예측
test_sentence = "I plays the piano."
test_encoding = tokenizer.encode_plus(
    test_sentence,
    add_special_tokens=True,
    max_length=max_length,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)
with torch.no_grad():
    model.eval()
    test_input_ids = test_encoding['input_ids'].flatten()
    test_attention_mask = test_encoding['attention_mask'].flatten()
    test_outputs = model(test_input_ids.unsqueeze(0), attention_mask=test_attention_mask.unsqueeze(0))
    predicted_label = torch.argmax(test_outputs.logits).item()



if predicted_label == 0:
    print("문법 오류가 있습니다.")
else:
    print("문법 오류가 없습니다.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1


##실제 문제 풀이

In [ ]:
def predict_problem(model_path, problem):
    # 모델 로드
    loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    loaded_model.load_state_dict(torch.load(model_save_path))
    loaded_model.eval()

    problem = list(map(lambda x: x.replace("[", "").replace("]", ""), problem))
    tokenized_sentences = [word_tokenize(sentence) for sentence in problem]
    sentences = []
    for tokenized_sentence in tokenized_sentences:
        sentences.append([TEXT.vocab.stoi[word] for word in tokenized_sentence])

    with torch.no_grad():
        classifier.eval()
        predict = []
        for sentence in sentences:
            sentence = torch.LongTensor([sentence])
            predict += [classifier(sentence).item()]
    answer = predict.index(min(predict)) +1
    print(f'정답은 {answer}번 입니다')

In [ ]:
def sat_pre(sentences):
  numbers =['①','②','③','④','⑤']

  sentences = sentences.replace('!','.').replace('?','.')
  sentences = sentences.split('.')
  problem = [sentence for sentence in sentences if any(number in sentence for number in numbers)]

  return problem

In [ ]:
def solving_problem(model_path,sentences):
  problem = sat_pre(sentences)

  return predict_problem(model_path,problem)

In [ ]:
problem_3 = '''The Greeks focus on the salient object and its attributes led to ①their failure to understand the fundamental nature of causality. Aristotle explained that a stone falling through the air is due to the stone having the property of "gravity". But of course a piece of wood ②tossed into waterfloats instead of sinking. This phenomenon Aristotle explained as being due to the wood having the property of"levity"! In both cases the focus is ③exclusively on the object, with no attention paid to the possibility thatsome force outside the object might be relevant. But the Chinese saw the world as consisting of continuously interacting substances, so their attempts to understand it ④causing them to be oriented toward the complexitiesof the entire "field," that is, the context or environment as a whole. The notion ⑤that events always accour in a field of forces would have been completely intuitive to the Chinese.'''


In [ ]:
solving_problem('/content/drive/MyDrive/미니캡스톤 자료/bert_grammar_model(20)두번째.pth',problem_3)

UnpicklingError: ignored

#책(파이토치 프로젝트)

##학습

In [1]:
!pip install dill

In [2]:
import torchtext

!pip install --upgrade torchtext==0.6.0


In [3]:
import dill
from copy import deepcopy
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc

import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

import torch

import torch.nn as nn


from torchtext.data import TabularDataset
from torchtext.data import BucketIterator
from torchtext.data import Iterator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from torchtext.data import Field

In [5]:
RANDOM_SEED = 2020
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# DATA_PATH = "data/processed/"
DATA_PATH = "/content/drive/MyDrive/미니캡스톤 자료"

In [6]:
import pandas as pd

# a=pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_test.tsv',sep='\t')
# a = a[['context','label']]
# a.to_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_test1.tsv',sep='\t',index=False)

In [ ]:
# b=pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_train.tsv',sep='\t')
# b = a[['context','label']]
# b.to_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_train1.tsv',sep='\t',index=False)

In [ ]:
# c=pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_valid.tsv',sep='\t')
# c = a[['context','label']]
# c.to_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_valid1.tsv',sep='\t',index=False)

In [9]:
# d=pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat.csv', sep=',', encoding='cp949')
# d = d[['context','label']]

# d.to_csv('/content/drive/MyDrive/미니캡스톤 자료/sat추가.tsv', sep='\t',index=False)

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# b=pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_train1.tsv',sep='\t')
# d = pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat추가.tsv',sep='\t')

# e = pd.concat([b,d], axis = 0)
# e.to_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_train2.tsv',sep='\t',index=False)

In [7]:
d=pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_train2.tsv',sep='\t',)
d.head()

,context,label
0,Competitive activities can be more than just p...,0
1,"The provision of timely, constructive feedback...",1
2,"In a sense, all competitions give feedback.",1
3,"For many, this is restricted to information ab...",1
4,The provision of that type of feedback can be ...,1


In [ ]:
e=pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/cola_train.tsv',sep='\t',)
e.head()

,context,label
0,"Our friends won't buy this analysis, let alone...",1
1,One more pseudo generalization and I'm giving up.,1
2,One more pseudo generalization or I'm giving up.,1
3,"The more we study verbs, the crazier they get.",1
4,Day by day the facts are getting murkier.,1


In [8]:
#어떻게 전처리를 진행할 것인지 정의
TEXT = Field(
    sequential=True,  #시퀀스 데이터인지
    use_vocab=True,   #단어 집합을 만들 것인지
    tokenize=word_tokenize, #어떤 토큰화 함수를 쓸 것인지
    lower=True,             #소문자화 할것인지
    batch_first=True,       #미니 배치 차원을 맨 앞으로 해서 데이터를 불러올 것인지
)
LABEL = Field(
    sequential=False,
    use_vocab=False,
    batch_first=True,
)

#데이터셋 만들기
#TabularDataset:  위 필드에서 정의했던 토큰화방법으로 토큰화 수행
cola_train_data, cola_valid_data, cola_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="cola_train.tsv",
    validation="cola_valid.tsv",
    test="cola_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1        #데이터 첫번째줄은 무시
)

#단어 집합 생성
TEXT.build_vocab(cola_train_data, min_freq=2)

#BucketIterator : 비슷한 길이의 문장들끼리 batch를 만들어 padding개수를 줄일 수 있음
cola_train_iterator, cola_valid_iterator, cola_test_iterator = BucketIterator.splits(
    (cola_train_data, cola_valid_data, cola_test_data),
    batch_size=32,
    device=None,
    sort=False,
)

#TabularDataset:  위 필드에서 정의했던 토큰화방법으로 토큰화 수행
sat_train_data, sat_valid_data, sat_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="sat_train2.tsv",
    validation="sat_valid1.tsv",
    test="sat_test1.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1
)

#BucketIterator : 비슷한 길이의 문장들끼리 batch를 만들어 padding개수를 줄일 수 있음
sat_train_iterator, sat_valid_iterator, sat_test_iterator = BucketIterator.splits(
    (sat_train_data, sat_valid_data, sat_test_data),
    batch_size=8,
    device=None,
    sort=False,
)

In [ ]:
# class LSTMPoolingClassifier(nn.Module):
#     def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
#         super(LSTMPoolingClassifier, self).__init__()
#         self.embed_layer = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=pad_idx)
#         self.hidden_size = hidden_size
#         self.embedding_dim = embedding_dim
#         self.num_layers = num_layers
#         self.ih2h = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
#                             bidirectional=True, batch_first=True, dropout=0.5)
#         self.pool2o = nn.Linear(2 * hidden_size, 1)
#         self.sigmoid = nn.Sigmoid()
#         self.softmax = nn.Softmax()
#         self.dropout = nn.Dropout(p=0.5)

#     def forward(self, x):
#         x = self.embed_layer(x)
#         o, _ = self.ih2h(x)
#         pool = nn.functional.max_pool1d(o.transpose(1, 2), x.shape[1])
#         pool = pool.transpose(1, 2).squeeze()
#         pool = self.dropout(pool)
#         output = self.sigmoid(self.pool2o(pool))
#         return output.squeeze()

In [9]:
class LSTMPoolingClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super(LSTMPoolingClassifier, self).__init__()
        self.embed_layer = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=pad_idx)

        self.ih2h= nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
                            bidirectional=True, batch_first=True, dropout=0.5)
        self.pool2o = nn.Linear(2 * hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout(p=0.5)
        self.last_layer = nn.Sequential(
            nn.Linear(2* hidden_size,1),
            nn.Dropout(p=0.5),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.embed_layer(x)
        o, _ = self.ih2h(x)
        pool = nn.functional.max_pool1d(o.transpose(1, 2), x.shape[1])
        pool = pool.transpose(1, 2).squeeze()
        output = self.last_layer(pool)
        return output.squeeze()

In [10]:
def train(model: nn.Module,
          iterator: Iterator,
          optimizer: torch.optim.Optimizer,
          criterion: nn.Module,
          device: str):
    model.train()
    epoch_loss = 0
    for _, batch in enumerate(iterator):
        optimizer.zero_grad()

        text = batch.text
        if text.shape[0] > 1:
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)

            output = model(text).flatten()
            loss = criterion(output, label)
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: Iterator,
             criterion: nn.Module,
             device: str):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def test(
    model: nn.Module,
    iterator: Iterator,
    device: str):

    with torch.no_grad():
        y_real = []
        y_pred = []
        model.eval()
        for batch in iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)

            output = model(text).flatten().cpu()

            y_real += [label]
            y_pred += [output]

        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)

    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc

def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [11]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]  #TEXT.vocab.stoi : 단어집합내 단어 확인
N_EPOCHS = 20

lstm_pool_classifier = LSTMPoolingClassifier(
    num_embeddings=len(TEXT.vocab),
    embedding_dim=100,
    hidden_size=200,
    num_layers=4,
    pad_idx=PAD_IDX,
)

if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
_ = lstm_pool_classifier.to(device)

optimizer = torch.optim.Adam(lstm_pool_classifier.parameters())
bce_loss_fn = nn.BCELoss()

In [12]:
for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(lstm_pool_classifier, cola_train_iterator, optimizer, bce_loss_fn, device)
    valid_loss = evaluate(lstm_pool_classifier, cola_valid_iterator, bce_loss_fn, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f}')
    print(f'\t Val. Loss: {valid_loss:.5f}')

Epoch: 01 | Time: 0m 3s
	Train Loss: 0.65142
	 Val. Loss: 0.64209
Epoch: 02 | Time: 0m 3s
	Train Loss: 0.64881
	 Val. Loss: 0.63039
Epoch: 03 | Time: 0m 2s
	Train Loss: 0.64596
	 Val. Loss: 0.63351
Epoch: 04 | Time: 0m 2s
	Train Loss: 0.63286
	 Val. Loss: 0.61844
Epoch: 05 | Time: 0m 2s
	Train Loss: 0.62432
	 Val. Loss: 0.62224
Epoch: 06 | Time: 0m 2s
	Train Loss: 0.61131
	 Val. Loss: 0.63177
Epoch: 07 | Time: 0m 3s
	Train Loss: 0.59930
	 Val. Loss: 0.61581
Epoch: 08 | Time: 0m 3s
	Train Loss: 0.58106
	 Val. Loss: 0.61197
Epoch: 09 | Time: 0m 2s
	Train Loss: 0.56045
	 Val. Loss: 0.62989
Epoch: 10 | Time: 0m 2s
	Train Loss: 0.54415
	 Val. Loss: 0.62440
Epoch: 11 | Time: 0m 2s
	Train Loss: 0.52921
	 Val. Loss: 0.63820
Epoch: 12 | Time: 0m 4s
	Train Loss: 0.50981
	 Val. Loss: 0.65760
Epoch: 13 | Time: 0m 4s
	Train Loss: 0.50185
	 Val. Loss: 0.68199
Epoch: 14 | Time: 0m 3s
	Train Loss: 0.48726
	 Val. Loss: 0.66932
Epoch: 15 | Time: 0m 2s
	Train Loss: 0.47234
	 Val. Loss: 0.69381
Epoch: 16 

In [13]:
before_tuning_lstm_pool_classifier = deepcopy(lstm_pool_classifier)

In [14]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20


for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(lstm_pool_classifier, sat_train_iterator, optimizer, bce_loss_fn, device)
    valid_loss = evaluate(lstm_pool_classifier, sat_valid_iterator, bce_loss_fn, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f}')
    print(f'\t Val. Loss: {valid_loss:.5f}')

Epoch: 01 | Time: 0m 0s
	Train Loss: 0.71479
	 Val. Loss: 0.63952
Epoch: 02 | Time: 0m 0s
	Train Loss: 0.58944
	 Val. Loss: 0.35314
Epoch: 03 | Time: 0m 0s
	Train Loss: 0.45879
	 Val. Loss: 0.34612
Epoch: 04 | Time: 0m 0s
	Train Loss: 0.48000
	 Val. Loss: 0.29324
Epoch: 05 | Time: 0m 0s
	Train Loss: 0.40974
	 Val. Loss: 0.26047
Epoch: 06 | Time: 0m 0s
	Train Loss: 0.37462
	 Val. Loss: 0.18917
Epoch: 07 | Time: 0m 0s
	Train Loss: 0.42267
	 Val. Loss: 0.16554
Epoch: 08 | Time: 0m 0s
	Train Loss: 0.41552
	 Val. Loss: 0.15323
Epoch: 09 | Time: 0m 0s
	Train Loss: 0.36064
	 Val. Loss: 0.15274
Epoch: 10 | Time: 0m 0s
	Train Loss: 0.39081
	 Val. Loss: 0.12112
Epoch: 11 | Time: 0m 0s
	Train Loss: 0.36259
	 Val. Loss: 0.10424
Epoch: 12 | Time: 0m 0s
	Train Loss: 0.27123
	 Val. Loss: 0.07052
Epoch: 13 | Time: 0m 0s
	Train Loss: 0.39804
	 Val. Loss: 0.04979
Epoch: 14 | Time: 0m 0s
	Train Loss: 0.36280
	 Val. Loss: 0.04011
Epoch: 15 | Time: 0m 0s
	Train Loss: 0.32671
	 Val. Loss: 0.03699
Epoch: 16 

In [15]:
_ = before_tuning_lstm_pool_classifier.cpu()
_ = lstm_pool_classifier.cpu()

pool_sat_test_auroc = test(before_tuning_lstm_pool_classifier, sat_test_iterator, "cpu")
pool_tuned_test_auroc = test(lstm_pool_classifier, sat_test_iterator, "cpu")

print(f"Before fine-tuning SAT Dataset Test AUROC: {pool_sat_test_auroc:.5f}")
print(f"After fine-tuning SAT Dataset Test AUROC: {pool_tuned_test_auroc:.5f}")

Before fine-tuning SAT Dataset Test AUROC: 0.46154
After fine-tuning SAT Dataset Test AUROC: 1.00000


In [16]:
with open("/content/drive/MyDrive/미니캡스톤 자료/advanced_after_tuning_model(두번째).dill", "wb") as f:
    model = {
        "TEXT": TEXT,
        "LABEL": LABEL,
        "classifier": lstm_pool_classifier
    }
    dill.dump(model, f)

##테스트

In [ ]:
import torchtext

!pip install --upgrade torchtext==0.6.0

In [ ]:
!pip install dill

In [ ]:
import dill
import random
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
import torch
import torch.nn as nn
import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

from torchtext.data import TabularDataset
from torchtext.data import BucketIterator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
RANDOM_SEED = 2020
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# DATA_PATH = "data/processed/"
DATA_PATH = "/content/drive/MyDrive/미니캡스톤 자료"

In [17]:
def predict_problem(model_path, problem):
    with open(model_path, "rb") as f:
        model = dill.load(f)
    TEXT = model["TEXT"]
    classifier = model["classifier"]

    problem = list(map(lambda x: x.replace("[", "").replace("]", ""), problem))
    tokenized_sentences = [word_tokenize(sentence) for sentence in problem]
    sentences = []
    for tokenized_sentence in tokenized_sentences:
        sentences.append([TEXT.vocab.stoi[word] for word in tokenized_sentence])

    with torch.no_grad():
        classifier.eval()
        predict = []
        for sentence in sentences:
            sentence = torch.LongTensor([sentence])
            predict += [classifier(sentence).item()]
    answer = predict.index(min(predict)) +1
    print(f'정답은 {answer}번 입니다')

In [18]:
problem_1 = [
    "Competitive activities can be more than just performance showcases which the best is recognized and the rest are overlooked.",
    "The provision of timely, constructive feedback to participants on performance is an asset that some competitions and contests offer.",
    "The provision of that type of feedback can be interpreted as shifting the emphasis to demonstrating superior performance but not necessarily excellence.",
    "The emphasis on superiority is what we typically see as fostering a detrimental effect of competition.",
    "Information about performance can be very helpful, not only to the participant who does not win or place but also to those who do.",
]
problem_1_label = [0, 1, 1, 1, 1]

In [19]:
predict_problem('/content/drive/MyDrive/미니캡스톤 자료/advanced_after_tuning_model(두번째).dill',problem_1)

정답은 1번 입니다


In [ ]:
problem_2 = [
    "People from more individualistic cultural contexts tend to be motivated to maintain self-focused agency or control 1 as these serve as the basis of one’s self-worth.",
    "With this form of agency comes the belief that individual successes 2 depending primarily on one’s own abilities and actions, and thus, whether by influencing the environment or trying to accept one’s circumstances, the use of control ultimately centers on the individual.",
    "The independent self may be more 3 driven to cope by appealing to a sense of agency or control.",
    "Research has shown 4 that East Asians prefer to receive, but not seek, more social support rather than seek personal control in certain cases.",
    "Therefore, people 5 who hold a more interdependent self-construal may prefer to cope in a way that promotes harmony in relationships.",
]
problem_2_label = [1, 0, 1, 1, 1]

In [ ]:
predict_problem('/content/drive/MyDrive/미니캡스톤 자료/advanced_after_tuning_model.dill',problem_2)

정답은 2번 입니다


##실제 문제 풀이

In [20]:
problem_3 = '''The Greeks focus on the salient object and its attributes led to ①their failure to understand the fundamental nature of causality. Aristotle explained that a stone falling through the air is due to the stone having the property of "gravity". But of course a piece of wood ②tossed into waterfloats instead of sinking. This phenomenon Aristotle explained as being due to the wood having the property of"levity"! In both cases the focus is ③exclusively on the object, with no attention paid to the possibility thatsome force outside the object might be relevant. But the Chinese saw the world as consisting of continuously interacting substances, so their attempts to understand it ④causing them to be oriented toward the complexitiesof the entire "field," that is, the context or environment as a whole. The notion ⑤that events always accour in a field of forces would have been completely intuitive to the Chinese.'''


In [21]:
def sat_pre(sentences):
  numbers =['①','②','③','④','⑤']

  sentences = sentences.replace('!','.').replace('?','.')
  sentences = sentences.split('.')
  problem = [sentence for sentence in sentences if any(number in sentence for number in numbers)]

  return problem

In [22]:
def predict_problem(model_path, problem):
    with open(model_path, "rb") as f:
        model = dill.load(f)
    TEXT = model["TEXT"]
    classifier = model["classifier"]

    problem = list(map(lambda x: x.replace("[", "").replace("]", ""), problem))
    tokenized_sentences = [word_tokenize(sentence) for sentence in problem]
    sentences = []
    for tokenized_sentence in tokenized_sentences:
        sentences.append([TEXT.vocab.stoi[word] for word in tokenized_sentence])

    with torch.no_grad():
        classifier.eval()
        predict = []
        for sentence in sentences:
            sentence = torch.LongTensor([sentence])
            predict += [classifier(sentence).item()]
    answer = predict.index(min(predict)) +1
    print(f'정답은 {answer}번 입니다')

In [28]:
def solving_problem(model_path,sentences):
  problem = sat_pre(sentences)

  return predict_problem(model_path,problem)

In [27]:
solving_problem('/content/drive/MyDrive/미니캡스톤 자료/advanced_after_tuning_model(두번째).dill',problem_3)

정답은 2번 입니다


#BERT(책)

In [ ]:
!pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.9 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=b4354527dae7a15d55a850e11d394158819c5431a0b83630a0c71a41addd3df0
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling u

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_train.tsv', sep='\t') #구글 드라이브에서 파일 경로 복사 후 붙여넣으세요.
valid_df = pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_valid.tsv', sep='\t') #구글 드라이브에서 파일 경로 복사 후 붙여넣으세요.
test_df = pd.read_csv('/content/drive/MyDrive/미니캡스톤 자료/sat_test.tsv', sep='\t') #구글 드라이브에서 파일 경로 복사 후 붙여넣으세요.

In [ ]:
test_df.head()

,context,label,year,test_type,question_number,question,answers
0,Competitive activities can be more than just p...,0,2021,9월,29,"다음 글의 밑줄 친 부분 중, 어법상 틀린 것은?",1
1,"The provision of timely, constructive feedback...",1,2021,9월,29,"다음 글의 밑줄 친 부분 중, 어법상 틀린 것은?",2
2,"In a sense, all competitions give feedback.",1,2021,9월,29,"다음 글의 밑줄 친 부분 중, 어법상 틀린 것은?",0
3,"For many, this is restricted to information ab...",1,2021,9월,29,"다음 글의 밑줄 친 부분 중, 어법상 틀린 것은?",0
4,The provision of that type of feedback can be ...,1,2021,9월,29,"다음 글의 밑줄 친 부분 중, 어법상 틀린 것은?",3


In [ ]:
# train_df = train_df.sample(frac=0.1, random_state=500)
# valid_df = valid_df.sample(frac=0.1, random_state=500)
# test_df = test_df.sample(frac=0.1, random_state=500)

In [ ]:
class Datasets(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]
        return text, label

In [ ]:
train_dataset = Datasets(train_df)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

valid_dataset = Datasets(valid_df)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True, num_workers=0)

test_dataset = Datasets(test_df)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=0)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

100%|██████████| 440473133/440473133 [00:09<00:00, 44604645.92B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
def save_checkpoint(save_path, model, valid_loss):
    if save_path == None:
        return
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):
    if load_path==None:
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):
    if save_path == None:
        return
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_metrics(load_path):
    if load_path==None:
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [ ]:
def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          num_epochs = 5,
          eval_every = len(train_loader) // 2,
          best_valid_loss = float("Inf")):

    total_correct = 0.0
    total_len = 0.0
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    model.train()
    for epoch in range(num_epochs):
        for text, label in train_loader:
            optimizer.zero_grad()
            encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
            padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]

            sample = torch.tensor(padded_list)
            sample, label = sample.to(device), label.to(device)
            labels = torch.tensor(label)
            outputs = model(sample, labels=labels)
            loss, logits = outputs

            pred = torch.argmax(F.softmax(logits), dim=1)
            correct = pred.eq(labels)
            total_correct += correct.sum().item()
            total_len += len(labels)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            global_step += 1

            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():
                    for text, label in valid_loader:
                        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
                        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
                        sample = torch.tensor(padded_list)
                        sample, label = sample.to(device), label.to(device)
                        labels = torch.tensor(label)
                        outputs = model(sample, labels=labels)
                        loss, logits = outputs
                        valid_running_loss += loss.item()

                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                running_loss = 0.0
                valid_running_loss = 0.0
                model.train()

                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))

                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint('/content/drive/MyDrive/미니캡스톤 자료/model.pt', model, best_valid_loss)
                    save_metrics('/content/drive/MyDrive/미니캡스톤 자료/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)

    save_metrics('/content/drive/MyDrive/미니캡스톤 자료/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('훈련 종료!')

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=2e-5)
train(model=model, optimizer=optimizer)

<ipython-input-9-0bd64172102c>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
<ipython-input-9-0bd64172102c>:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)
<ipython-input-9-0bd64172102c>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)


Epoch [1/5], Step [6/60], Train Loss: 0.4429, Valid Loss: 0.3779
Model saved to ==> /content/drive/MyDrive/미니캡스톤 자료/model.pt
Model saved to ==> /content/drive/MyDrive/미니캡스톤 자료/metrics.pt
Epoch [1/5], Step [12/60], Train Loss: 0.4021, Valid Loss: 0.3728
Model saved to ==> /content/drive/MyDrive/미니캡스톤 자료/model.pt
Model saved to ==> /content/drive/MyDrive/미니캡스톤 자료/metrics.pt
Epoch [2/5], Step [18/60], Train Loss: 0.5281, Valid Loss: 0.3781
Epoch [2/5], Step [24/60], Train Loss: 0.3030, Valid Loss: 0.3743
Epoch [3/5], Step [30/60], Train Loss: 0.4523, Valid Loss: 0.3626
Model saved to ==> /content/drive/MyDrive/미니캡스톤 자료/model.pt
Model saved to ==> /content/drive/MyDrive/미니캡스톤 자료/metrics.pt
Epoch [3/5], Step [36/60], Train Loss: 0.4113, Valid Loss: 0.3725
Epoch [4/5], Step [42/60], Train Loss: 0.3043, Valid Loss: 0.3734
Epoch [4/5], Step [48/60], Train Loss: 0.5317, Valid Loss: 0.6047
Epoch [5/5], Step [54/60], Train Loss: 0.4168, Valid 

In [ ]:
train_loss_list, valid_loss_list, global_steps_list = load_metrics('/content/drive/MyDrive/Colab Notebooks/metrics.pt')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.plot(global_steps_list, valid_loss_list, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def evaluate(model, test_loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for text, label in test_loader:
            encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
            padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]

            sample = torch.tensor(padded_list)
            sample, label = sample.to(device), label.to(device)
            labels = torch.tensor(label)
            output = model(sample, labels=labels)

            _, output = output
            y_pred.extend(torch.argmax(output, 1).tolist())
            y_true.extend(labels.tolist())

    print('Classification 결과:')
    print(classification_report(y_true, y_pred, labels=[1,0], digits=4))

    cm = confusion_matrix(y_true, y_pred, labels=[1,0])
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")

    ax.set_title('Confusion Matrix')
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')
    ax.xaxis.set_ticklabels(['0', '1'])
    ax.yaxis.set_ticklabels(['0', '1'])

In [ ]:
import warnings
warnings.filterwarnings('ignore')
best_model = model.to(device)
load_checkpoint('/content/drive/MyDrive/Colab Notebooks/model.pt', best_model)
evaluate(best_model, test_loader)

In [ ]:
def sat_pre(sentences):
  numbers =['①','②','③','④','⑤']

  sentences = sentences.replace('!','.').replace('?','.')
  sentences = sentences.split('.')
  problem = [sentence for sentence in sentences if any(number in sentence for number in numbers)]

  return problem

In [ ]:
sentences = 'Most historians of science point to the need for a reliable calendar to regulate agricultural activity as the motivation for learning about what we now call astronomy, the study of stars and planets. Early astronomy provided information about when
to plant crops and gave humans ① their first formal method of
recording the passage of time. Stonehenge, the 4,000-year-old
ring of stones in southern Britain, ② is perhaps the best-known
monument to the discovery of regularity and predictability in
the world we inhabit. The great markers of Stonehenge point to
the spots on the horizon ③ where the sun rises at the solstices
and equinoxes ― the dates we still use to mark the beginnings
of the seasons. The stones may even have ④ been used to
predict eclipses. The existence of Stonehenge, built by people
without writing, bears silent testimony both to the regularity of
nature and to the ability of the human mind to see behind
immediate appearances and ⑤ discovers deeper meanings in
events.'

SyntaxError: ignored

In [ ]:
sentences = sentences.replace('!','.').replace('?','.')
sentences = sentences.split('.')
problem = [sentence for sentence in sentences if any(number in sentence for number in numbers)]

problem
